# Übung 2 Detektion von Verkehrsschildern

**German Traffic Sign Detection Benchmark**

Detallierte Beschreibung des Datensatzes siehe unter folgendem [Link](http://benchmark.ini.rub.de/?section=gtsdb&subsection=news)

## Imports

In [22]:
import os

import csv
import wget #wget是一个从网络上自动下载文件的自由工具。它支持HTTP，HTTPS和FTP协议，可以使用HTTP代理。

import cv2
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual, widgets

In [23]:
# Testfunktion für ipywidgets: 
# Es soll ein Slider angezeigt werden. Der Wertebereich des Sliders
# soll zwischen -10(min) und 30(max) liegen. 
# Entsprechend der Sliderposition soll ein Ergebniswert angezeigt werden.
def f(x):
    return 3 * x
interact(f, x= 10);

interactive(children=(IntSlider(value=10, description='x', max=30, min=-10), Output()), _dom_classes=('widget-…

## Globale Variablen

Um hartcodierte Bezeichner/Namen in den Funktionen zu vermeiden, definiere an dieser Stelle alle Variablen, die global verwendet werden.

In [24]:
# Definiere den Pfad zum heruntergeladenen Datenordner
DATA_PATH = 'C:/Users/hqpet/Desktop/BildAuto_Hausaufgabe/UE2/'

# Prüfe, ob der Pfad existiert / korrekt eingegeben wurde
assert os.path.exists(DATA_PATH), "Der angegebene Pfad existriert nicht."

In [25]:
# Definiere den Pfad zur Datei gt.txt
"""
TIPP: arbeite zuerst mit der Datei new_gt.txt, um den Rechenaufwand bei dem Aufruf 
der Funktion calculate_hough_cirles() zu reduzieren.
Diese Datei enthält wenige Bilder mit meist gut sichtbaren Verkehrszeichen 

"""
### TO DO ###
# Definiere den Pfad zur gt.txt-Datei / zur new_gt.txt-Datei
GT_TXT_PATH = os.path.join(DATA_PATH, 'new_gt.txt')
assert os.path.exists(GT_TXT_PATH), "Der angegebene Pfad existriert nicht."

In [26]:
# Prohibitory Class IDs 
PROHIBITORY_CLASS_IDs = [ 0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 15, 16]

# Mandatory Class IDs
MANDATORY_CLASS_IDs = [ 33, 34, 35, 36, 37, 38, 39, 40 ]

# Danger Class IDs
DANGER_CLASS_IDs = [ 11, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31 ]

In [27]:
# MANDATORY-Dictionary 
MANDATORY_DICT = {}
# MANDATORY images (filenames) list
MANDATORY_IMG_LIST = []
# MANDATORY filepaths
MANDATORY_FILEPATHS = []

In [28]:
def calculate_mandatory(): 
    """
    Sortiert Verkehrszeichen nach der Kategorie „mandatory“ und speichert 
    die Ergebnisse in ein Dictionary.
    Das Dictionary beinhaltet Dateinamen als Schlüssel und Listen von 
    Ground Thruth ROIs-Listen als Values.
    """
    global MANDATORY_DICT
    # Setze prohibitory dictionary zurück
    MANDATORY_DICT.clear()
    
    # Öffne die gt.txt-Datei / new_gt.txt-Datei
    with open(GT_TXT_PATH, newline='') as csvfile:
        gt_reader = csv.reader(csvfile, delimiter=';')#gt_reader中存储的是列表类型数据，为了方便理解我们就将gt_reader看成是个列表嵌套列表的对象
        
        # Bau eine Schleife, um die Daten Zeile für Zeile auszulesen 
        # und die entsprechende Liste der ROIs für die Datei zu füllen
        for row in gt_reader: #row是一个列表
            ### TO DO ###
            key = str(row[0])
            if key not in MANDATORY_DICT:
                 MANDATORY_DICT[key] = []
            MANDATORY_DICT[key].append(list(map(int,row[1:6])))

## Aufgabe 1 - Aussortieren bestimmter Verkehrsschilder

In [29]:
# Ermittele die Dateienamen (ausgehend von DATA_PATH) alle Treffer in MANDATORY_DICT
# MANDATORY_FILEPATHS = os.path.splitext(DATA_PATH)[0]
MANDATORY_FILEPATHS = [os.path.join(DATA_PATH, 'FullIJCNN2013', key) for key in MANDATORY_DICT.keys()]
print(len(MANDATORY_FILEPATHS))

0


In [30]:
# Funktionsaufruf
"""
Erwartete Ausgabe:
    {'00002.ppm': [[892, 476, 1006, 592, 39]],
     '00039.ppm': [[953, 252, 1015, 313, 35]],
     '00040.ppm': [[1040, 310, 1130, 400, 33]],
     '00066.ppm': [[569, 483, 618, 535, 38]],
     '00074.ppm': [[383, 527, 421, 569, 38]],
     '00081.ppm': [[233, 499, 249, 521, 38]],
     ...}
"""
calculate_mandatory()
print(MANDATORY_DICT)

{'00002.ppm': [[892, 476, 1006, 592, 39]], '00005.ppm': [[1172, 164, 1284, 278, 9]], '00010.ppm': [[1193, 358, 1269, 436, 12]], '00011.ppm': [[720, 338, 770, 398, 1], [710, 265, 780, 342, 21]], '00013.ppm': [[1043, 281, 1120, 356, 2]], '00023.ppm': [[998, 292, 1074, 371, 1], [46, 350, 136, 444, 23], [945, 435, 1006, 492, 23]], '00039.ppm': [[938, 167, 1022, 251, 12], [953, 252, 1015, 313, 35]], '00040.ppm': [[1040, 310, 1130, 400, 33], [1017, 202, 1141, 313, 13]], '00046.ppm': [[1045, 126, 1166, 247, 12]], '00066.ppm': [[569, 483, 618, 535, 38]], '00074.ppm': [[383, 527, 421, 569, 38], [439, 401, 488, 452, 14], [929, 378, 981, 431, 14]], '00078.ppm': [[1211, 206, 1319, 305, 29], [800, 462, 821, 483, 13]], '00081.ppm': [[966, 228, 1044, 299, 13], [233, 499, 249, 521, 38]], '00083.ppm': [[1054, 208, 1152, 296, 13], [1071, 297, 1142, 368, 40]], '00098.ppm': [[1206, 244, 1296, 323, 13], [594, 286, 668, 353, 13]], '00114.ppm': [[1169, 421, 1231, 484, 3]], '00115.ppm': [[850, 218, 950, 317, 

In [31]:
def render_mandatory_rois():
    """
    Malt die ROIs (Rechtecke) auf die entsprechenden Bilder und speichert 
    die in MANDATORY_IMG_LIST. 
    Hinweis:
    Die ROIs und Bildernamen können aus MANDATORY_DICT ermittelt werden
    """
    # Setze die globale variable zurück
    MANDATORY_IMG_LIST.clear()
    
    for key in MANDATORY_DICT.keys():
        file_path = os.path.join(DATA_PATH,"FullIJCNN2013", key)
        img = plt.imread(file_path)
        ### TO DO ###
        for idx in range(len(MANDATORY_DICT[key])):
            # Berechne Koordinaten des Rechtecks  矩形对角线的点
            point1 = (MANDATORY_DICT[key][idx][0],MANDATORY_DICT[key][idx][1])
            point2 = (MANDATORY_DICT[key][idx][2],MANDATORY_DICT[key][idx][3])
            
            # Zeichne das Rechteck
            img = cv2.rectangle(img,point1,point2,(0,255,0),2)
            org =  (MANDATORY_DICT[key][idx][2] + 10, MANDATORY_DICT[key][idx][1]+20)          
            # Speichere Verkehrszeichennamen als text
            text = str(MANDATORY_DICT[key][-1])
            img = cv2.putText(img,text,org,cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),5)
            #putText(img, text, org, fontFace, fontScale, color, thickness=None, lineType=None, bottomLeftOrigin=None):
            #图片，要添加的文字，文字添加到图片上的位置，字体的类型，字体大小，字体颜色，字体粗细
            
        MANDATORY_IMG_LIST.append(img)

In [32]:
# Mandatory Image list abrufen
render_mandatory_rois()
print(len(MANDATORY_IMG_LIST))
# print(MANDATORY_IMG_LIST)

98


In [33]:
def show_img(idx):
    plt.figure(figsize=(16,8))
    plt.imshow(MANDATORY_IMG_LIST[idx])
    plt.show()

In [34]:
interact(show_img, idx=widgets.IntSlider(min=0,max=len(MANDATORY_IMG_LIST)-1, step=1, value=0));

interactive(children=(IntSlider(value=0, description='idx', max=97), Output()), _dom_classes=('widget-interact…

## Aufgabe 2 – Formbasierter Ansatz


Hier kannst du die Links abrufen, die zum Implementieren dieser Funktion nützlich sein können:

- https://docs.opencv.org/4.x/da/d53/tutorial_py_houghcircles.html
- https://docs.opencv.org/4.x/df/d9d/tutorial_py_colorspaces.html
- https://docs.opencv.org/4.x/d4/d13/tutorial_py_filtering.html 

Übersicht der Tutorials: https://docs.opencv.org/4.x/d6/d00/tutorial_py_root.html

In [35]:
def calculate_hough_cirles(filepaths, d_p, min_dist, param1, param2, min_radius, max_radius):
    """
     Berechnet Hough Circles unter Berücksichtigung der Form der Verkehrszeichen
    """
    
    # Liste fuer die Speicherung des Ergebnis
    result = []
    predicted_dict = {}
    
    for filepath_ in filepaths:
        # Lade das Bild in color_img
        color_img = cv2.imread(filepath_, cv2.IMREAD_COLOR)
        ### TO DO ###
        
        # Konvertiere das BGR-Bild in Gray.
        # https://docs.opencv.org/4.x/df/d9d/tutorial_py_colorspaces.html
        img_gray = cv2.cvtColor(color_img, cv2.COLOR_BGR2GRAY)

        # Reduziere das Rauschen 
        # https://docs.opencv.org/4.x/d4/d13/tutorial_py_filtering.html --> Kapitel "Image bluring"
        img_blurred = cv2.blur(img_gray,(5,5))
        
        # Ermittele die Kreisen auf dem Bild 
        # https://docs.opencv.org/4.x/da/d53/tutorial_py_houghcircles.html
        circles = cv2.HoughCircles(img_blurred,cv2.HOUGH_GRADIENT, dp=d_p, minDist=min_dist, 
                                  param1=param1, param2=param2, minRadius=min_radius, maxRadius=max_radius)
#         print(type(circles))

        # Kreise auf das Bild malen
        if circles is not None: 
            # Kreise-Paramater in interger umwandeln
            circles = np.uint16(np.around(circles)) 
            #np.around()函数对输入浮点数执行5舍6入，5做特殊处理（小数点最后一位为5的舍入为与其值最接近的偶数值）。
            
            # Kreise auf das Bild malen
            for point in circles[0, :]: 
                a, b, r = point[0], point[1], point[2]#color_img.shape
                cv2.circle(img_blurred,(a,b),r,(0,255,0),2)
                # Ermittle Koordinaten der Rechtecke, die für die Evaluation benutzt werden
                point1 = a-r,b-r
                point2 = a+r,b+r
                # OPTIONAL: Rechtecke auf das Bild malen
                cv2.rectangle(color_img,point1,point2,(0,255,0),2)
                if os.path.split(filepath_)[-1] in predicted_dict:
                    if a==0 and b==0 and r==0:
                        continue
                    predicted_dict[os.path.split(filepath_)[-1]].append([point1[0], point1[1], point2[0], point2[1]])
                else: 
                    predicted_dict[os.path.split(filepath_)[-1]] = []
                    if a==0 and b==0 and r==0:
                        continue
                    predicted_dict[os.path.split(filepath_)[-1]].append([point1[0], point1[1], point2[0], point2[1]])
        else:
            predicted_dict[os.path.split(filepath_)[-1]] = []                
        result.append(color_img)
        
    return result, predicted_dict

In [36]:
pred_imgs_form, predicted_rect_rois = calculate_hough_cirles(MANDATORY_FILEPATHS, 
                                                  d_p=1,
                                                  min_dist=150, 
                                                  param1= 100, 
                                                  param2= 40, 
                                                  min_radius=5, 
                                                  max_radius=80) 

In [37]:
def show_img_form(idx):
    plt.figure(figsize=(16,8))
    plt.imshow(cv2.cvtColor(pred_imgs_form[idx], cv2.COLOR_BGR2RGB))
    plt.show()

In [38]:
interact(show_img_form, idx=widgets.IntSlider(min=0, max=len(pred_imgs_form)-1, step=1, value=0));

TraitError: setting max < min

## Aufgabe 3 – Optimierung und Evaluation des formbasierten Ansatzes
对基于形式的方法进行优化和评估

In [ ]:
def jaccard_similarity(pred, gr_truth):
    '''
    Berechnet den Jaccard-Koeffizienten für zwei Rechtecke: den vorhergesagen (pred) und den ground_truth (gr_truth)
    Jaccard系数（Jaccard index），又称为Jaccard相似系数（Jaccard similarity coefficient）用于比较有限样本集之间的相似性与差异性。
    Jaccard系数：给定两个集合A和B，Jaccard系数定义为A与B交集的大小与A与B并集的大小的比值
    J(A,B)∈[0,1]。Jaccard系数值越大，样本相似度越高。当集合A，B都为空时，J(A,B)定义为1。
    Jaccard 距离：用于描述集合之间的不相似度。
    '''
    # Ermittle die (x, y)-Koordinaten der Schnittmenge beider Rechtecke x,y指的哪个矩形的坐标
    x_i1 = max(pred[0], gr_truth[0])
    y_i1 = max(pred[1], gr_truth[1])
    x_i2 = min(pred[2], gr_truth[2])
    y_i2 = min(pred[3], gr_truth[3])

    inter_area = max(0, x_i2 - x_i1 + 1) * max(0, y_i2 - y_i1 + 1)

    pred_area = (pred[2] - pred[0] + 1) * (pred[3] - pred[1] + 1)
    gr_truth_area = (gr_truth[2] - gr_truth[0] + 1) * (gr_truth[3] - gr_truth[1] + 1)
    
    iou = inter_area / float(pred_area + gr_truth_area - inter_area)
    
    # Gebe den "Intersection Over Union"-Wert zurück
    return iou

In [ ]:
def evaluate_detection(ground_truth_dict, predicted_dict, similarity_threshold=0.6):    
    '''
    Evaluiert implementierte Ansätze anhand des Jaccard-Ähnlichkeitsmaßes
    Referenz für die Berechnung: Houben et. al. Kapitel IV Evaluation Procedure
    '''
    # True Positives
    tp = 0
    # False Positives
    fp = 0
    # False Negatives
    fn = 0
    
    for key in ground_truth_dict.keys():
        # Liste mit allen ROIs eines Dateinamens 
        rois_gt_lists = ground_truth_dict[key]
        
        # Berechne Jaccard-Ähnlichkeitsmaß von detektierten Rechtecken, die aus den Kreiskoordinaten ermittelt wurden
        rois_pred_lists = predicted_dict[key]
        
        if len(rois_pred_lists) > 0:
            for rois_gt_list in rois_gt_lists:
                iou = [jaccard_similarity(rois_pred, rois_gt_list) for rois_pred in rois_pred_lists]
                
                # Liste mit den Werten, die kleiner als similarity_threshold sind
                iou_lt_threshold = [value for value in iou if value < similarity_threshold]
                fp = fp + len(iou_lt_threshold)
                
                # Liste mit den Werten, die größer / gleich similarity_threshold sind
                iou_gt_threshold = [value for value in iou if value >= similarity_threshold]
                
                if len(iou_gt_threshold) > 0 : 
                    tp = tp + 1
                else:
                    fn = fn + 1
        else:
            fn = fn + len(rois_gt_lists)

    return tp, fp, fn            

In [ ]:
def calculate_precision_recall(tp, fp, fn):
    '''
    Berechnet Precision- und Recall-Werte
    '''
    
    precision = math.nan #Python math.nan 返回一个浮点值 nan（not a number，不是一个数字），这个值不是一个合法数字。
    # precision = float('nan') math.nan 相当于 float('nan') 的输出。
    if tp + fp != 0:
        precision = tp / (tp + fp)   
    
    recall = math.nan
    # recall = float('nan')
    if tp + fn != 0:
        recall = tp / (tp + fn)
        
    return precision, recall

In [ ]:
# similarity_threshold entspricht dem Schwellenwert im Paper von Houben et. al.
# tp_form, fp_form, fn_form = evaluate_detection(MANDATORY_DICT, predicted_rect_rois, similarity_threshold=0.6)

In [ ]:
# Precision-Recall-Plot
### TO DO ###

In [ ]:
precision_list = []
recall_list = []
for value in range(100,40,-2):
    pred_imgs_form, predicted_rect_rois = calculate_hough_cirles(MANDATORY_FILEPATHS, 
                                                  d_p=1,
                                                  min_dist=150, 
                                                  param1= 100, 
                                                  param2= value, 
                                                  min_radius=5, 
                                                  max_radius=80) 
    tp_form,fp_form,fn_form = evaluate_detection(MANDATORY_DICT,predicted_rect_rois,similarity_threshold=0.6)
    precision,recall =calculate_precision_recall(tp_form,fp_form,fn_form)
    precision_list.append(precision)
    recall_list.append(recall)
    
plt.figure()
plt.plot(recall_list,precision_list,marker='.',label='Hough Circle')
plt.title('PR-curve')
plt.xlabel('recall')
plt.ylabel('precision')
plt.legend()
plt.show()